In [2]:
%matplotlib qt
from mpl_toolkits import mplot3d
import numpy as np
import matplotlib.pyplot as plt
import time

import matplotlib.animation as animation
import networkx as nx
from matplotlib.animation import FuncAnimation, PillowWriter

# 2+1D Transport Equation

In [8]:
# Numerical Parameters

# Grid Points and Time Steps
N = int(1e2+1) # +1 for Off-By-One on Periodic BC
M = int(1e2+1)
T = int(5e2+1)
# Domain Size
L = 10
K = 10
# Domain
x = np.linspace(0, L, N)
y = np.linspace(0, K, M)
# Grid Size and Time Step Size
dx = x[2] - x[1]
dy = y[2] - y[1]
dt = 1e-3

lmbdx = dt/dx
lmbdy = dt/dy

# Coordinate Plane
xx,yy = np.meshgrid(x,y)
t = dt * T

In [9]:
# Memory Allocation
u = np.zeros((T,M,N))

# Inital Condition
rho_0 = 1
freq = 2*np.pi
freq_x = 2*freq/L
freq_y = 3*freq/K
uIC = rho_0 + 2*np.sin(freq_x*xx)+np.sin(freq_y*yy)
u[0] = np.copy(uIC)

Fx = 30
Fy = 20

print(lmbdx*Fx+lmbdy*Fy)

0.5


In [4]:
# # Solve
# st = time.time()
# for tt in range(0,T-1):
#     for jj in range(0,M):
#         for ii in range(0,N):
#             u[tt+1,jj,ii] = u[tt,jj,ii] - lmbdy*Fy*(u[tt,jj,ii]-u[tt,jj-1,ii]) - lmbdx*Fx*(u[tt,jj,ii]-u[tt,jj,ii-1])
#     if tt % ((T-1)/10) == 0:
#         print(tt)
# et = time.time()
# elapsed_time = et - st
# print('Execution time:', elapsed_time, 'seconds')

# Take 2 Cause I'm Bored On  A Plane
# Step 1: Vectorization

In [3]:
def left(array):
    return np.roll(array, -1, axis = -1)
    
def right(array):
    return np.roll(array, 1, axis = -1)
    
def up(array):
    return np.roll(array, -1, axis = -2)
    
def down(array):
    return np.roll(array, 1, axis = -2)
    
def yFlux(array):
    return Fy*(array-down(array))

def xFlux(array):
    return Fx*(array-left(array))

def d2dx2(array):
    return right(array) - 2*array + left(array)

def d2dy2(array):
    return down(array) - 2*array + up(array)

In [11]:
# Solve
st = time.time()
for tt in range(0,T-1):
    u[tt+1] = u[tt] - lmbdy*yFlux(u[tt]) - lmbdx*xFlux(u[tt])
    if tt % ((T-1)/10) == 0:
        print(tt)

et = time.time()
elapsed_time = et - st
print('Execution time:', elapsed_time, 'seconds')

0
50
100
150
200
250
300
350
400
450
Execution time: 0.06268811225891113 seconds


In [12]:
#Plotting
def plot(utt):
    fig = plt.figure(figsize=(15,15))
    ax = plt.axes(projection='3d')
    # ax.contour3D(xx, yy, uIC, 50, cmap='binary')
    # for elm in u:
    #     ax.contour3D(xx, yy, elm, 500, cmap='binary')
    ax.contour3D(xx,yy,utt,100)
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_zlabel('z')
    plt.show()

In [6]:
# #Animation
# def change_plot(frame_number,u, plot):
#    plot[0].remove()
#    plot[0] = ax.plot_surface(xx, yy, u[frame_number,:,:], cmap="viridis")
    
# fig = plt.figure()
# ax = fig.add_subplot(111, projection='3d')

# plot = [ax.plot_surface(xx, yy, u[0, :, :], color='0.75', rstride=10, cstride=10)]

# ani = animation.FuncAnimation(fig, change_plot, T, fargs=(u, plot), interval= 1)
# # ani.save(filename=r"C:\Users\Chris Gerlach\OneDrive\Desktop\PDEmultidimension.gif", writer='pillow',fps=30)

# plt.show()

# 2+1D Heat Equation

In [ ]:
# Numerical Parameters

# Grid Points and Time Steps
N = int(1e2+1) # +1 for Off-By-One on Periodic BC
M = int(1e2+1)
T = int(5e4+1)
# Domain Size
L = 10
K = 10
# Domain
x = np.linspace(0, L, N)
y = np.linspace(0, K, M)
# Grid Size and Time Step Size
dx = x[2] - x[1]
dy = y[2] - y[1]
dt = 1e-4

mux = dt/(dx*dx)
muy = dt/(dy*dy)

# Coordinate Plane
xx,yy = np.meshgrid(x,y)
t = dt * T

In [ ]:
# Memory Allocation
v = np.zeros((T,M,N))

v[0] = np.exp(-(xx-L/2)**2-(yy-K/2)**2)

Fx = 10
Fy = 20

print(dx*dy/(4*(Fx*Fy)))

In [ ]:
# Plot Initial Condition

# fig = plt.figure()
# ax = fig.add_subplot(111, projection='3d')
# plot = [ax.plot_surface(xx, yy, v[0], color='0.75', rstride=10, cstride=10)]
# plt.show()

In [ ]:
# Solve
st = time.time()
for tt in range(0,T-1):
    v[tt+1] = v[tt] + muy*Fy*d2dy2(v[tt]) + mux*Fx*d2dx2(v[tt])
           
    if tt % ((T-1)/10) == 0:
        print(tt)
et = time.time()
elapsed_time = et - st
print('Execution time:', elapsed_time, 'seconds')

In [ ]:
#Animation
def change_plot(frame_number,v, plot):
   plot[0].remove()
   plot[0] = ax.plot_surface(xx, yy, v[frame_number,:,:], cmap="viridis")
    
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

plot = [ax.plot_surface(xx, yy, v[0, :, :], color='0.75', rstride=10, cstride=10)]
ax.set_zlim(0,1.5)

ani = animation.FuncAnimation(fig, change_plot, T, fargs=(v, plot), interval= 100, save_count=1000)

# plt.show()

In [ ]:
ani.save(filename=r"C:\Users\chris\OneDrive\Desktop\PDEmultidimension_heat.gif", writer='pillow',fps=30)

# 2+1D Advection/Diffusion Equation

In [3]:
# Numerical Parameters

# Grid Points and Time Steps
N = int(1e2+1) # +1 for Off-By-One on Periodic BC
M = int(1e2+1)
T = int(1e4+1)
# Domain Size
L = 10
K = 10
# Domain
x = np.linspace(0, L, N)
y = np.linspace(0, K, M)
# Grid Size and Time Step Size
dx = x[2] - x[1]
dy = y[2] - y[1]
dt = 1e-4

lmbdx = dt/dx
lmbdy = dt/dy

mux = dt/(dx*dx)
muy = dt/(dy*dy)

# Coordinate Plane
xx,yy = np.meshgrid(x,y)
t = dt * T

In [4]:
def left(array):
    return np.roll(array, -1, axis = -1)
    
def right(array):
    return np.roll(array, 1, axis = -1)
    
def up(array):
    return np.roll(array, -1, axis = -2)
    
def down(array):
    return np.roll(array, 1, axis = -2)
    
def ddx(array, dx):
    return (array-down(array))/dx

def ddy(array, dy):
    return Fx*(array-left(array))/dy

def d2dx2(array, dx):
    return (right(array) - 2*array + left(array))/(dx**2)

def d2dy2(array, dy):
    return (down(array) - 2*array + up(array))/(dy**2)

In [5]:
# Memory Allocation
w = np.zeros((T,M,N))

# Inital Condition
rho_0 = 1
freq = 2*np.pi
freq_x = 2*freq/L
freq_y = 3*freq/K
wIC = rho_0 + 2*np.sin(freq_x*xx)+np.sin(freq_y*yy)
w[0] = np.copy(wIC)

Fx = 3
Fy = 2

Dx = 1
Dy = 2

print(lmbdx*Fx+lmbdy*Fy)

0.005


In [6]:
# Solve
st = time.time()
for tt in range(0,T-1):
    w[tt+1] = w[tt] + dt*(Dy*d2dy2(w[tt], dy) + Fy*ddy(w[tt], dy) + Dx*d2dx2(w[tt], dx) + Fx*ddx(w[tt], dx))
           
    if tt % ((T-1)/10) == 0:
        print(tt)
et = time.time()
elapsed_time = et - st
print('Execution time:', elapsed_time, 'seconds')

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
Execution time: 5.2413976192474365 seconds


In [7]:
#Animation
def change_plot(frame_number,data, plot):
   plot[0].remove()
   plot[0] = ax.plot_surface(xx, yy, data[frame_number,:,:], cmap="viridis")
    
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

plot = [ax.plot_surface(xx, yy, w[0, :, :], color='0.75', rstride=10, cstride=10)]

ani = animation.FuncAnimation(fig, change_plot, T, fargs=(w, plot), interval= 10, save_count=10)

# plt.show()

In [ ]:
ani.save(filename=r"C:\Users\chris\OneDrive\Desktop\PDEmultidimension_advection_diffusion.gif", writer='pillow',fps=30)